In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import time
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn import metrics

In [2]:
df=pd.read_csv("adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','captial gain','capital loss','hours per week','native country','Salaray'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,captial gain,capital loss,hours per week,native country,Salaray
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
def removeQuestionMark(outCol,inpCol):
    for i in inpCol:
        repl=X[i].value_counts().keys().tolist()[0]
        X[i]=X[i].replace(to_replace=' ?',value=repl)
    rep=y[outCol[0]].value_counts().keys().tolist()[0]
    y[outCol[0]]=y[outCol[0]].replace(to_replace=' ?',value=rep)
    
def Splitting(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return [X_train,y_train,X_test,y_test]

def CategTraining(X_train,y_train,outCol):
    trainCount=y_train[outCol[0]].value_counts().to_dict()
    col1=list(X_train.columns)
    outputList=y_train[outCol[0]].value_counts().keys().tolist()
    trainDict=dict([(key, []) for key in col1])
    for i in range(0,len(col1)):
        inputList=X_train[col1[i]].value_counts().keys().tolist()

        proxy=dict([(key, dict([(keys, []) for keys in outputList])) for key in inputList])
        trainDict[col1[i]]=proxy
    for i in trainDict.keys():
        for j in trainDict[i].keys():
            for k in trainDict[i][j].keys():
                num=(X_train.loc[(X_train[i]==j) &  (y_train[outCol[0]]==k)]).shape[0]
                den=trainCount[k]
                prob=num/den
                trainDict[i][j][k]=prob
    return trainDict

def CategTesting(outCol,trainDict,y_train,y_test,X_test):
    trainProb=(y_train[outCol[0]].value_counts()/y_train.shape[0]).to_dict()
    outputList=y_train[outCol[0]].value_counts().keys().tolist()
    testDict1=dict([(key,dict([(keys, []) for keys in outputList])) for key in y_test.index])
    for i in testDict1.keys():
        for j in testDict1[i].keys():
            prob=1
            l=0
            for k in trainDict.keys():
                prob=trainDict[k][X_test.loc[i][l]][j]*prob
                l=l+1
            testDict1[i][j]=prob*trainProb[j]
    return testDict1

def CategPredict(testDict,y_test):
    size=X_test.shape[0]
    predict=dict((key,[]) for key in y_test.index)
    for i in predict.keys():
        maxi=0
        l=''
        for j in testDict[i].keys():
            if(testDict[i][j]>maxi):
                maxi=testDict[i][j]
                l=j
        predict[i]=l
    accuracy=0
    count=0
    for i in predict.keys():
        if(y_test.loc[i][0]==predict[i]):
            count=1+count
    accuracy=count*100/size
    print(accuracy)

def Probop(y_test,posOp):
    Probop=dict([(key,[]) for key in posOp])
    for i in Probop.keys():
        Probop[i]=(y_test['Salaray'].value_counts()[i])/y_test.shape[0]
    return Probop

def ContTraining(X_train,y_train):
    NumericDict=dict([(key,dict([(keys, []) for keys in posOp])) for key in X_train.columns])
    Inplist=dict([(key,dict([(keys, []) for keys in posOp])) for key in X_train.columns])
    for i in NumericDict.keys():
        for j in NumericDict[i].keys():
            Inplist[i][j]=(y_train.loc[y_train['Salaray']==j]).index.tolist()
    for i in NumericDict.keys():
        for j in NumericDict[i].keys():
            count=len(Inplist[i][j])
            su=0
            for k in range(0,count):
                su+=(X_train.loc[Inplist[i][j][k]])[i]
            NumericDict[i][j]=[su/count]
    for i in Inplist.keys():
        for j in Inplist[i].keys():
            count=len(Inplist[i][j])
            diff=0
            for k in range(0,count):
                diff+=((X_train.loc[Inplist[i][j][k]])[i]-NumericDict[i][j][0])**2
            NumericDict[i][j].append(diff/count)
    return NumericDict    

def ContTesting(X_test,posOp,NumericDict):
    testInd=X_test.index
    prediction=dict([(key,dict([(keys,dict([(key,[]) for key in posOp])) for keys in testInd])) for key in X_test.columns])
    for i in prediction.keys():
        for j in posOp:
            den=math.sqrt(2*math.pi*(NumericDict[i][j][1]**2))
            count=len(prediction[i])
            for k in range(0,count):
                num=math.exp(-(X_test.loc[testInd[k]][i]-NumericDict[i][j][0]/(2*math.pow(NumericDict[i][j][1],2))))
                prediction[i][testInd[k]][j]=num/den
    Predict=dict([(key,dict([(keys, []) for keys in posOp])) for key in y_test.index])
    for i in Predict.keys():
        for j in posOp:
            prob=1
            for k in prediction.keys():
                if(prediction[k][i][j]!=0):
                    prob=prob*prediction[k][i][j]
            Predict[i][j]=prob
    return Predict

def ContPredict(prediction,ProbOp,y_test):
    testInd=y_test.index
    FinalPrediction=dict([(keys,[]) for keys in testInd])
    for i in testInd:
        maxi=0
        l=''
        for j in posOp:
            prob=1
            for k in prediction.keys():
                if(prediction[k][i][j]!=0):
                    prob=prob*prediction[k][i][j]
            prob=prob*ProbOp[j]
            if(prob>maxi):
                maxi=prob
                l=j
        FinalPrediction[i]=l 
    acc=0
    count=0
    length=len(testInd)
    for i in range(0,length):
            if(FinalPrediction[testInd[i]]==y_test.iloc[i,0]):
                count+=1
    acc=count*100/y_test.shape[0]
    print(acc)
    
def Predict(testDict,prediction,ProbOp,y_test):
    testInd=y_test.index
    FinalPrediction=dict([(keys,[]) for keys in testInd])
    for i in FinalPrediction.keys():
        maxi=0
        pr=''
        p=0
        for j in posOp:
            p=prediction[i][j]*testDict[i][j]
            if(p>maxi):
                maxi=p
                pr=j
        FinalPrediction[i]=pr
    acc=0
    count=0
    length=len(testInd)
    for i in range(0,length):
            if(FinalPrediction[testInd[i]]==y_test.iloc[i,0]):
                count+=1
    acc=count*100/y_test.shape[0]
    return acc

In [4]:
start=time.time()
X=df[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country','Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
y=df[['Salaray']]
[x_train,y_train,x_test,y_test]=Splitting(X,y)
inpCol=list(X.columns)
outCol=list(y.columns)
removeQuestionMark(outCol,inpCol)
X_train=x_train[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
X_test=x_test[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
trainDict=CategTraining(X_train,y_train,outCol)
testDict=CategTesting(outCol,trainDict,y_train,y_test,X_test)
X_train=x_train[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
X_test=x_test[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
posOp=y_train['Salaray'].value_counts().keys().tolist()
size=y_train.shape[0]
ProbOp=Probop(y_test,posOp)
NumericDict=ContTraining(X_train,y_train)
prediction=ContTesting(X_test,posOp,NumericDict)
Predict(testDict,prediction,ProbOp,y_test)
print(time.time()-start)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


129.71632432937622


In [6]:
X=df[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country','Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
y=df[['Salaray']]
[x_train,y_train,x_test,y_test]=Splitting(X,y)
inpCol=list(X.columns)
outCol=list(y.columns)
removeQuestionMark(outCol,inpCol)
[x_train,y_train,x_test,y_test]=Splitting(X,y)
X_train=x_train[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
X_test=x_test[['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
trainDict=CategTraining(X_train,y_train,outCol)
testDict=CategTesting(outCol,trainDict,y_train,y_test,X_test)
X_train=x_train[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
X_test=x_test[['Age','fnlwgt','education num','captial gain','capital loss','hours per week']]
posOp=y_train['Salaray'].value_counts().keys().tolist()
size=y_train.shape[0]
ProbOp=Probop(y_test,posOp)
clf = GaussianNB()
clf.fit(X_train,y_train)
s=clf.predict_proba(X_test)
r=0
c=0
prediction=dict([(key,dict([(keys, []) for keys in posOp])) for key in y_test.index])
for i in prediction.keys():
    c=0
    for j in prediction[i].keys():
        prediction[i][j]=s[r][c]
        c=c+1
    r=r+1
print("The accuracy of the algorithm in use is "+str(Predict(testDict,prediction,ProbOp,y_test)))

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y

The accuracy of the algorithm in use is 81.42082096427475


In [7]:
trainDict

{'Workplace': {' Private': {' <=50K': 0.783203011873733,
   ' >50K': 0.6562330378143658},
  ' Self-emp-not-inc': {' <=50K': 0.07471763683753258,
   ' >50K': 0.0915505699294373},
  ' Local-gov': {' <=50K': 0.058905299739357084, ' >50K': 0.07743803148181654},
  ' State-gov': {' <=50K': 0.038459310744280335, ' >50K': 0.04613714492491406},
  ' Self-emp-inc': {' <=50K': 0.02038806834636548,
   ' >50K': 0.08051384114347747},
  ' Federal-gov': {' <=50K': 0.023342021430640025,
   ' >50K': 0.04812737470598878},
  ' Without-pay': {' <=50K': 0.0005792064871126556, ' >50K': 0.0},
  ' Never-worked': {' <=50K': 0.00040544454097885896, ' >50K': 0.0}},
 'education': {' HS-grad': {' <=50K': 0.35708079930495223,
   ' >50K': 0.20879319703274832},
  ' Some-college': {' <=50K': 0.23880683463654792,
   ' >50K': 0.17586394065496652},
  ' Bachelors': {' <=50K': 0.12748334781349552, ' >50K': 0.2842409987334901},
  ' Masters': {' <=50K': 0.030987547060527077, ' >50K': 0.12285145648633979},
  ' Assoc-voc': {' <=

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NaiveBayes(priors=None, var_smoothing=1e-09)

0.7993653393387246